<a href="https://colab.research.google.com/github/shruteeegrg/Machine-Learning-with-Python/blob/main/Natural_Language_Processing_(NLP)_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Natural Language Processing**



###Word Embeddings
Luckily there is a third method that is far superior, **word embeddings**. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beggining of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.

This is the technique we will use for our examples and its implementation will be showed later on.



##Recurrent Neural Networks (RNN's)
Now that we've learned a little bit about how we can encode text it's time to dive into recurrent neural networks. Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exaclty what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

Let's have a look at what a recurrent layer might look like.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Let's define what all these variables stand for before we get into the explination.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

What this diagram is trying to illustrate is that a recurrent layer processes words or input one at a time in a combination with the output from the previous iteration. So, as we progress further in the input sequence, we build a more complex understanding of the text as a whole.

What we've just looked at is called a **simple RNN layer**. It can be effective at processing shorter sequences of text for simple problems but has many downfalls associated with it. One of them being the fact that as text sequences get longer it gets increasingly difficult for the network to understand the text properly.



# **Sentiment Analysis**

The example we’ll use here is classifying movie reviews as either postive, negative or neutral.

###Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.





In [17]:
from keras.datasets import imdb
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584 #number of unique words in the dataset

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

###More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

Luckily for us keras has a function that can do this for us:




In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# **Training**

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - acc: 0.6714 - loss: 0.5745 - val_acc: 0.8474 - val_loss: 0.3625
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - acc: 0.8855 - loss: 0.2911 - val_acc: 0.8656 - val_loss: 0.3177
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - acc: 0.9183 - loss: 0.2179 - val_acc: 0.8864 - val_loss: 0.2858
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - acc: 0.9369 - loss: 0.1780 - val_acc: 0.8852 - val_loss: 0.2917
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - acc: 0.9464 - loss: 0.1525 - val_acc: 0.8718 - val_loss: 0.3578
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - acc: 0.9542 - loss: 0.1315 - val_acc: 0.8452 - val_loss: 0.4254
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - acc: 0.9609 - loss: 0.1154 - val_acc: 0.8824 - val_loss: 0.3194
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - acc: 0.9679 - loss: 0.0971 - val_acc: 0.8778 - val_loss: 0.3400
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 1

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - acc: 0.8659 - loss: 0.4352
[0.43216419219970703, 0.8669999837875366]


###Making Predictions
Now let’s use our network to make predictions on our own reviews.

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
[0.96320784]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[0.43666515]


##RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.

In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

**Dataset**

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving donald_speech.txt to donald_speech (1).txt


In [14]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 7233 characters


In [15]:
# Take a look at the first 250 characters in text
print(text[:250])

Chief Justice Roberts, President Carter, President Clinton,President Bush, fellow Americans and people of the world – thank you.
 We the citizens of America have now joined a great national effort to rebuild our county and restore its promise for all


# **Data(Text) Preprocessing**

**Encoding**

In [16]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)} #  Maps characters to unique integers for numerical processing.
idx2char = np.array(vocab) # Maps integers back to characters for interpretation.

def text_to_int(text): #Converts text into a sequence of integers
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [17]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: Chief Justice
Encoded: [12 37 38 34 35  1 18 49 47 48 38 32 34]


In [18]:
def int_to_text(ints): #Converts a sequence of integers back into text
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

Chief Justice


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [20]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [21]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [22]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [23]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
Chief Justice Roberts, President Carter, President Clinton,President Bush, fellow Americans and peop

OUTPUT
hief Justice Roberts, President Carter, President Clinton,President Bush, fellow Americans and peopl


EXAMPLE

INPUT
e of the world – thank you.
 We the citizens of America have now joined a great national effort to r

OUTPUT
 of the world – thank you.
 We the citizens of America have now joined a great national effort to re


In [45]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024



# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# **Till Now **

**Text Preparation:** Convert text into integers (text_as_int) using a
character-to-integer mapping (char2idx).


**Dataset Creation:** Slice the text into overlapping sequences of seq_length + 1. Split each sequence into input (chunk[:-1]) and target (chunk[1:]).


**Shuffling and Batching:** Shuffle the dataset and batch it for efficient processing.


**Preview: **Display a few examples of input and target sequences for verification.

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [93]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  output_dim=embedding_dim),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                            stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# Create and summarize the model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.build(input_shape=(BATCH_SIZE, None))
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (64, None, 256)             │          14,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (64, None, 1024)            │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (64, None, 57)              │          58,425 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,319,993 (20.29 MB)

 Trainable params: 5,319,993 (20.29 MB)

 Non-trainable params: 0 (0.00 B)

**Loss Function**

In [47]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 57) # (batch_size, sequence_length, vocab_size)


In [48]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.36007986e-03 -1.75965601e-03  3.35158757e-03 ...  1.20171264e-03
   -3.51308589e-03  5.63882757e-04]
  [ 3.13011277e-03 -9.26247798e-03  4.38501779e-03 ...  6.81867450e-03
   -3.05166328e-03  3.08305025e-04]
  [ 7.32728932e-03 -6.27743918e-03  6.00784272e-03 ...  2.41119880e-03
    1.50581915e-03 -4.19375894e-04]
  ...
  [ 7.06717372e-03 -5.31218573e-03  1.27437981e-02 ... -3.57707986e-03
   -1.13893766e-02 -7.57401064e-03]
  [ 1.04479678e-02 -1.87186757e-04  1.17481090e-02 ... -9.62538552e-03
   -1.35576343e-02 -1.09656984e-02]
  [ 1.30485222e-02 -1.09965252e-02  1.17783351e-02 ... -1.10044098e-03
   -1.15460064e-02 -9.19505954e-03]]

 [[-1.81229867e-03 -2.96424306e-03  6.20325934e-03 ...  1.55292405e-03
   -2.26863846e-03 -1.85384450e-03]
  [-3.74086434e-03  5.79491374e-04  4.02550306e-03 ...  4.76503605e-03
   -6.04779692e-04 -5.89495385e-03]
  [-6.39997330e-03  1.18718133e-03  9.93929058e-03 ... -4.81163012e-03
    2.87089078e-03 -1.89033500e-03]
  ...
  [ 1.357

In [49]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00136008 -0.00175966  0.00335159 ...  0.00120171 -0.00351309
   0.00056388]
 [ 0.00313011 -0.00926248  0.00438502 ...  0.00681867 -0.00305166
   0.00030831]
 [ 0.00732729 -0.00627744  0.00600784 ...  0.0024112   0.00150582
  -0.00041938]
 ...
 [ 0.00706717 -0.00531219  0.0127438  ... -0.00357708 -0.01138938
  -0.00757401]
 [ 0.01044797 -0.00018719  0.01174811 ... -0.00962539 -0.01355763
  -0.0109657 ]
 [ 0.01304852 -0.01099653  0.01177834 ... -0.00110044 -0.01154601
  -0.00919506]], shape=(100, 57), dtype=float32)


In [50]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

57
tf.Tensor(
[-1.3600799e-03 -1.7596560e-03  3.3515876e-03  8.0102583e-04
  7.7466713e-05  1.3442382e-03  2.8489917e-03  2.7384958e-03
 -6.1876066e-03  1.7571752e-03 -9.0066774e-04 -2.1560641e-04
  2.4279212e-03 -5.9413398e-03  2.5860821e-03  3.5289879e-05
  1.3540229e-03 -1.1602581e-03 -4.0231049e-03 -2.2267723e-03
 -6.7195501e-03 -5.5178050e-03 -9.9606905e-04 -2.2033714e-03
  5.0659552e-03  1.5918114e-03  6.9813908e-04  8.9617970e-04
 -2.1153267e-03 -2.6656890e-03 -1.0666442e-03  1.4483719e-04
 -5.7286602e-03  9.6562812e-03 -4.7888709e-03  6.1466254e-04
  2.3577670e-03  3.6341054e-03  2.0417299e-03 -1.0875039e-03
 -2.7550105e-03  1.2876547e-04  6.2090326e-03 -4.7579831e-03
  1.9507739e-04  4.4438150e-03 -4.2685317e-03 -3.7137223e-03
  5.4567535e-03  2.8260830e-03  5.4128664e-03 -5.2491971e-04
  3.1864243e-03 -2.4426682e-03  1.2017126e-03 -3.5130859e-03
  5.6388276e-04], shape=(57,), dtype=float32)


In [51]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"Jey0aP’gOWjcoIioSBUoWw2Dtz2Ua cI1y\n1\nUS–O’DWh–’CFjb2JhbdA1gl2D’–'PgoY0SdoUmn1Uux7wdldJ–wNDgte.Mzswp "

In [52]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [94]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [95]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files (must end with .weights.h5)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

# Define the callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True  # Ensure only weights are saved
)


**Training**

In [96]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 4.0417
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 3.9708
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 3.3578
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 3.5059
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 3.3996
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 3.1710
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - loss: 3.0299
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - loss: 3.0920
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 3.0209
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 3.0179
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 3.0149
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 3.0028
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 2.9821
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - loss: 2.9633
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 2.9366
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/s

**Load the Model**

In [97]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [98]:
# Ensure checkpoint directory exists
checkpoint_dir = './training_checkpoints'

# Verify checkpoints are saved and find the latest one
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print("Restoring from:", latest_checkpoint)
    model.load_weights(latest_checkpoint)
else:
    print("No checkpoint found. Starting from scratch.")

# Build the model with the expected input shape
model.build(tf.TensorShape([1, None]))


No checkpoint found. Starting from scratch.


In [99]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print("Restoring from:", latest_checkpoint)
    model.load_weights(latest_checkpoint)
else:
    print("No checkpoint found. Starting from scratch.")

# Build the model with the expected input shape
model.build(tf.TensorShape([1, None]))


No checkpoint found. Starting from scratch.


**Generating The Text**

In [100]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 800

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))